
### data formatting and engineering based on the data sample shared by S2A

- Extract all audios from video
- Convert all the text transcript from audios 
- return the size of the video and extracted audio files 
- Extract the topics and subtopics from meta data PPT
- Organize the data into the labeld data format in tabular format

In [4]:
# ! pip install SpeechRecognition moviepy

In [19]:
# pip install pydub

In [93]:
# https://stackoverflow.com/questions/3844430/how-to-get-the-duration-of-a-video-in-python

# pip install ffmpeg-python

In [107]:
# pip install SoundFile

In [20]:
import speech_recognition as sr 
import moviepy.editor as mp
import pandas as pd
import numpy as np
import glob
import os

In [230]:
list_of_videos = glob.glob('../../Data/videos_univ_chicago/*.mp4')
list_of_ppts =  glob.glob('../../Data/text_univ_chichago/*.pptx')

list_of_ppts = [x for x in list_of_ppts if "~$" not in x]

In [14]:
list_of_videos

['../../Data/videos_univ_chicago/birds_intro_flamingo_v.mp4',
 '../../Data/videos_univ_chicago/birds_intro_peacock_v.mp4',
 '../../Data/videos_univ_chicago/birds_intro_tailor_bird_v.mp4',
 '../../Data/videos_univ_chicago/insects_intro_ant_v.mp4',
 '../../Data/videos_univ_chicago/insects_intro_beetle_v.mp4']

Extrat Stats from Videos files

In [24]:
# get video files size on the video in Bytes

def file_size(vid_file):
    import os
    byts = os.path.getsize(vid_file)
    # then we return the bytes into bytes, metabyte, gb
    return byts, np.round(byts/1000000, 3), np.round(byts/1000000000, 3)

video_size = [file_size(i) for i in list_of_videos]
video_size

[(255680222, 255.68, 0.256),
 (49122115, 49.122, 0.049),
 (29499314, 29.499, 0.029),
 (137598834, 137.599, 0.138),
 (21366835, 21.367, 0.021)]

In [103]:
# get video files durations 
def video_duration(vid_file):
    from moviepy.editor import VideoFileClip
    clip = VideoFileClip(vid_file)
    return clip.duration,  np.round(clip.duration/60, 3)

In [104]:
video_duration = [video_duration(i) for i in list_of_videos]
video_duration

[(184.36, 3.073),
 (102.06, 1.701),
 (201.51, 3.358),
 (225.66, 3.761),
 (52.25, 0.871)]

convert Video into Audio files 

In [15]:
# Function to read all the video clips and convert to audio files 
def convert_to_audio(vd,  output_pth):
    clip = mp.VideoFileClip(vd) 
    ad_nm = vd.split("/")[-1].split(".")[0]
    clip.audio.write_audiofile(output_pth + "/" + ad_nm + ".wav")
    print("Audio {} has been successfully extracted in {}".format(ad_nm, output_pth))

In [17]:
# convert all video clips into Audio files 
for vid in list_of_videos:
    convert_to_audio(vd = vid, output_pth = 'Audios')

MoviePy - Writing audio in Audios/birds_intro_flamingo_v.wav


MoviePy - Done.
Audio birds_intro_flamingo_v has been successfully extracted in Audios
MoviePy - Writing audio in Audios/birds_intro_peacock_v.wav


MoviePy - Done.
Audio birds_intro_peacock_v has been successfully extracted in Audios
MoviePy - Writing audio in Audios/birds_intro_tailor_bird_v.wav


MoviePy - Done.
Audio birds_intro_tailor_bird_v has been successfully extracted in Audios
MoviePy - Writing audio in Audios/insects_intro_ant_v.wav


MoviePy - Done.
Audio insects_intro_ant_v has been successfully extracted in Audios
MoviePy - Writing audio in Audios/insects_intro_beetle_v.wav


MoviePy - Done.
Audio insects_intro_beetle_v has been successfully extracted in Audios


In [25]:
# audio files size 
list_of_audios = glob.glob('Audios/*.wav')
audio_size = [file_size(i) for i in list_of_audios]
audio_size

[(39806502, 39.807, 0.04),
 (35546442, 35.546, 0.036),
 (18003462, 18.003, 0.018),
 (9216978, 9.217, 0.009),
 (32521182, 32.521, 0.033)]

In [26]:
list_of_audios

['Audios/insects_intro_ant_v.wav',
 'Audios/birds_intro_tailor_bird_v.wav',
 'Audios/birds_intro_peacock_v.wav',
 'Audios/insects_intro_beetle_v.wav',
 'Audios/birds_intro_flamingo_v.wav']

Convert Audios ino Text files 

In [209]:
# Reading Large Audio Files
# If you want to perform speech recognition of a long audio file, then the below function handles that quite well:
# https://www.thepythoncode.com/article/using-speech-recognition-to-convert-speech-to-text-python

# read all the audo files from the path
# Convert audio into text files 

r = sr.Recognizer()
audio = sr.AudioFile("converted.wav")

# importing libraries 
import speech_recognition as sr 
import os 
from pydub import AudioSegment 
from pydub.silence import split_on_silence 

# create a speech recognition object
r = sr.Recognizer()

# a function that splits the audio file into chunks
# and applies speech recognition
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    fl_nm = path.split("/")[-1].split(".")[0]
    res_df = pd.DataFrame({"video_file" : [str(fl_nm)] ,  "raw_txt": [str(whole_text)] } )
    # return the text for all chunks detected and a dataframe that contain the text chunk
    return whole_text, res_df

# convert the convertsed txt into .txt files 
def txt_file(raw_txt,  text_output):
    with open(text_output, mode ='w') as file: 
        file.write("Recognized Speech:") 
        file.write("\n") 
        file.write(raw_txt) 
        print("text ready!")
    

In [210]:
audio_rawtxt = pd.DataFrame()
for audio in list_of_audios:
    print(audio + "\n")
    extracted_txt = get_large_audio_transcription(path = audio)
    txt_file(raw_txt = extracted_txt[0], text_output = "Audio_Text/" + path.split("/")[-1].replace(".wav", ".txt") )
    temp = extracted_txt[1]
    audio_rawtxt = pd.concat([audio_rawtxt, temp], axis = 0)

Audios/insects_intro_ant_v.wav

Error: 
Error: 
audio-chunks/chunk3.wav : Did you find an ant hill. 
audio-chunks/chunk4.wav : The app that you see outside or only a small number compared to all the answers below the soil. 
audio-chunks/chunk5.wav : These huge groups of ants are called calmese a single colony may contain thousands even millions of ants every ant in a colony has a specific job worker answer small ants that are responsible for everything from digging and repairing the tunnels to gathering food to caring for the young ants. 
audio-chunks/chunk6.wav : All the worker ants are females. 
Error: 
audio-chunks/chunk8.wav : The me last job is the fertilized the queen they are larger than the worker ass but not as large as the queen. 
audio-chunks/chunk9.wav : The largest and most important as of the colony is the queen. 
audio-chunks/chunk10.wav : What job is still latex there are four stages in an ant's life cycle. 
audio-chunks/chunk11.wav : The queen lays her eggs after a few

audio-chunks/chunk18.wav : Flamingos are filter feeders. 
audio-chunks/chunk19.wav : To eat. 
audio-chunks/chunk20.wav : They stand in shallow water and hang their heads down into it. 
audio-chunks/chunk21.wav : They sweep their heads from side-to-side with their bill below the surface. 
audio-chunks/chunk22.wav : And use their spiny town to help them strain food from the water. 
audio-chunks/chunk23.wav : Flamingos eat brine shrimp. 
audio-chunks/chunk24.wav : Algae. 
audio-chunks/chunk25.wav : Insects. 
audio-chunks/chunk26.wav : Mollusks and small crustaceans. 
audio-chunks/chunk27.wav : Because their diet includes both plants and animals. 
audio-chunks/chunk28.wav : Flamingos for omnivores you may be interested to know that flamingos are not born pink. 
audio-chunks/chunk29.wav : When they are chicks they're actually gray or white. 
audio-chunks/chunk30.wav : It takes a young flamingo two or three years before they attain their pink color they acquire their color gradually because 

In [212]:
# read all the txt files and format whole chunk of them into dataframe with video file name as unique identifier

audio_rawtxt.head()

,video_file,raw_txt
0,insects_intro_ant_v,Did you find an ant hill. The app that you see...
0,birds_intro_tailor_bird_v,This is a girl named masha. She's sleeping and...
0,birds_intro_peacock_v,Wow look at those feathers. The bird you found...
0,insects_intro_beetle_v,Beetles have a hard shell like armor. And i sa...
0,birds_intro_flamingo_v,Let's learn about flamingos flamingos are tall...


In [213]:
audio_rawtxt.to_csv("Targeted_Output/Phase1/0_raw_audio_rawtxt.csv", index = False)

Annotate audio files with timestamp

https://towardsdatascience.com/speech-recognition-with-timestamps-934ede4234b2

In [111]:
# pip install vosk


In [115]:
# pip install words

In [180]:
# source of code: https://towardsdatascience.com/speech-recognition-with-timestamps-934ede4234b2
# amke sure to install and set up correct file/model structure before run this code
def extract_timestamp_by_words_audio(path):
    import wave
    import json

    from vosk import Model, KaldiRecognizer, SetLogLevel
    import Word as custom_Word

    model_path = "models/vosk-model-en-us-0.21"
    audio_filename = path

    model = Model(model_path)
    wf = wave.open(audio_filename, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())
    rec.SetWords(True)

    # get the list of JSON dictionaries
    results = []
    # recognize speech using vosk model
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            part_result = json.loads(rec.Result())
            results.append(part_result)
    part_result = json.loads(rec.FinalResult())
    results.append(part_result)

    # convert list of JSON dictionaries to list of 'Word' objects
    list_of_Words = []
    for sentence in results:
        if len(sentence) == 1:
            # sometimes there are bugs in recognition 
            # and it returns an empty dictionary
            # {'text': ''}
            continue
        for obj in sentence['result']:
            w = custom_Word.Word(obj)  # create custom Word object
            list_of_Words.append(w)  # and add it to list

    wf.close()  # close audiofile

    # output to the screen
    # for word in list_of_Words:
    #     print(word.to_string())
    # output to a data frame 
    audio_word_time_df = pd.DataFrame(columns=['words', 'timestamps_audio'])
    for word in list_of_Words:
        word_timestamp_raw = {}
        lst =  word.to_string().split(" ")
        word_timestamp_raw[lst[0]] = " ".join([value for value in lst if value != ''][1:])
#         print(word_timestamp_raw)
        temp_df = pd.DataFrame(word_timestamp_raw.items(), columns=['words', 'timestamps_audio'])
        audio_word_time_df = pd.concat([audio_word_time_df, temp_df] , ignore_index=True, axis = 0 )

    audio_word_time_df['audio_source'] = audio_filename.split('/')[-1].split(".")[0]

    return audio_word_time_df

In [184]:
audio_word_timestamp = pd.DataFrame()
for audio_path in list_of_audios:
    temp = extract_timestamp_by_words_audio(path = audio_path)
    audio_word_timestamp = pd.concat([audio_word_timestamp, temp], axis = 0)

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (VoskAPI:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (VoskAPI:CompileLooped():nnet-compile-looped.cc:345) Spent 0.634587 seconds in looped compilation.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from models/vosk-model-en-us-0.21/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:278) Loading HCLG from models/vosk-model-en-us-0.21/graph/HCLG.fst
LOG (VoskAPI:ReadDataFiles():model.cc:293) Loading words from models/vosk-model-en-

{'oh': 'from 14.37 sec to 15.12 sec, confidence is 84.38%'}
{'cool': 'from 31.44 sec to 32.25 sec, confidence is 64.88%'}
{'hey': 'from 40.50 sec to 41.04 sec, confidence is 100.00%'}
{'you': 'from 58.88 sec to 59.10 sec, confidence is 51.12%'}
{'want': 'from 59.10 sec to 59.43 sec, confidence is 51.12%'}
{'your': 'from 59.43 sec to 59.55 sec, confidence is 51.12%'}
{'phone': 'from 59.55 sec to 60.00 sec, confidence is 51.12%'}
{'to': 'from 60.00 sec to 60.30 sec, confidence is 51.12%'}
{'me': 'from 60.36 sec to 61.11 sec, confidence is 100.00%'}
{'adults': 'from 62.16 sec to 63.15 sec, confidence is 30.47%'}
{'snow': 'from 73.38 sec to 74.28 sec, confidence is 57.85%'}
{'salt': 'from 74.34 sec to 75.09 sec, confidence is 98.15%'}
{'or': 'from 75.18 sec to 75.51 sec, confidence is 55.16%'}
{'snow': 'from 76.50 sec to 77.82 sec, confidence is 100.00%'}
{'mobile': 'from 77.85 sec to 78.66 sec, confidence is 92.90%'}
{'right': 'from 79.68 sec to 79.92 sec, confidence is 53.61%'}
{'spring'

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (VoskAPI:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (VoskAPI:CompileLooped():nnet-compile-looped.cc:345) Spent 0.647299 seconds in looped compilation.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from models/vosk-model-en-us-0.21/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:278) Loading HCLG from models/vosk-model-en-us-0.21/graph/HCLG.fst
LOG (VoskAPI:ReadDataFiles():model.cc:293) Loading words from models/vosk-model-en-

{'desi': 'from 1.44 sec to 1.98 sec, confidence is 42.69%'}
{'said': 'from 1.98 sec to 2.49 sec, confidence is 39.44%'}
{'can': 'from 2.49 sec to 3.27 sec, confidence is 75.04%'}
{'live': 'from 3.84 sec to 4.41 sec, confidence is 46.53%'}
{'marshall': 'from 4.65 sec to 5.70 sec, confidence is 52.18%'}
{'she': 'from 6.99 sec to 7.41 sec, confidence is 95.77%'}
{'is': 'from 7.41 sec to 7.80 sec, confidence is 80.83%'}
{'too': 'from 7.80 sec to 8.22 sec, confidence is 53.20%'}
{'thin': 'from 8.22 sec to 8.85 sec, confidence is 100.00%'}
{'out': 'from 9.60 sec to 9.89 sec, confidence is 61.07%'}
{'the': 'from 9.89 sec to 10.16 sec, confidence is 100.00%'}
{'wazoo': 'from 10.26 sec to 11.07 sec, confidence is 61.07%'}
{'is': 'from 11.55 sec to 11.91 sec, confidence is 97.72%'}
{'trying': 'from 11.91 sec to 12.63 sec, confidence is 100.00%'}
{'to': 'from 12.66 sec to 12.98 sec, confidence is 100.00%'}
{'reach': 'from 12.99 sec to 13.38 sec, confidence is 38.34%'}
{'her': 'from 13.47 sec to 1

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (VoskAPI:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (VoskAPI:CompileLooped():nnet-compile-looped.cc:345) Spent 0.647389 seconds in looped compilation.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from models/vosk-model-en-us-0.21/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:278) Loading HCLG from models/vosk-model-en-us-0.21/graph/HCLG.fst
LOG (VoskAPI:ReadDataFiles():model.cc:293) Loading words from models/vosk-model-en-

{'now': 'from 0.72 sec to 1.68 sec, confidence is 72.64%'}
{'let': 'from 1.68 sec to 2.07 sec, confidence is 96.78%'}
{'that': 'from 2.07 sec to 2.34 sec, confidence is 100.00%'}
{'pass': 'from 2.34 sec to 3.06 sec, confidence is 96.78%'}
{'ouch': 'from 3.75 sec to 4.74 sec, confidence is 36.83%'}
{'the': 'from 7.20 sec to 7.53 sec, confidence is 100.00%'}
{"you've": 'from 13.68 sec to 13.89 sec, confidence is 50.63%'}
{'seen': 'from 13.89 sec to 14.34 sec, confidence is 50.63%'}
{'also': 'from 17.97 sec to 18.84 sec, confidence is 92.49%'}
{'known': 'from 18.84 sec to 19.68 sec, confidence is 92.49%'}
{'as': 'from 19.83 sec to 20.22 sec, confidence is 100.00%'}
{'it': 'from 20.22 sec to 20.46 sec, confidence is 89.14%'}
{'girl': 'from 22.05 sec to 22.71 sec, confidence is 44.19%'}
{'there': 'from 24.06 sec to 25.20 sec, confidence is 87.82%'}
{'surf': 'from 26.40 sec to 26.87 sec, confidence is 41.44%'}
{'the': 'from 26.87 sec to 27.00 sec, confidence is 85.16%'}
{'new': 'from 27.00 s

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (VoskAPI:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (VoskAPI:CompileLooped():nnet-compile-looped.cc:345) Spent 0.649431 seconds in looped compilation.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from models/vosk-model-en-us-0.21/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:278) Loading HCLG from models/vosk-model-en-us-0.21/graph/HCLG.fst
LOG (VoskAPI:ReadDataFiles():model.cc:293) Loading words from models/vosk-model-en-

{'big': 'from 0.66 sec to 1.05 sec, confidence is 59.27%'}
{'chains': 'from 1.05 sec to 1.71 sec, confidence is 100.00%'}
{'have': 'from 1.74 sec to 2.04 sec, confidence is 100.00%'}
{'a': 'from 2.04 sec to 2.16 sec, confidence is 100.00%'}
{'whole': 'from 2.16 sec to 3.06 sec, confidence is 100.00%'}
{'show': 'from 3.09 sec to 4.17 sec, confidence is 100.00%'}
{'mark': 'from 4.92 sec to 5.46 sec, confidence is 61.34%'}
{'a': 'from 5.55 sec to 6.18 sec, confidence is 51.85%'}
{'ball': 'from 6.21 sec to 6.72 sec, confidence is 29.62%'}
{'a': 'from 8.64 sec to 8.75 sec, confidence is 61.98%'}
{'browse': 'from 8.75 sec to 9.24 sec, confidence is 100.00%'}
{'through': 'from 9.24 sec to 9.81 sec, confidence is 100.00%'}
{'search': 'from 10.17 sec to 10.68 sec, confidence is 100.00%'}
{'they': 'from 10.95 sec to 11.19 sec, confidence is 56.24%'}
{'have': 'from 11.19 sec to 11.55 sec, confidence is 72.18%'}
{'six': 'from 11.58 sec to 12.60 sec, confidence is 100.00%'}
{'loans': 'from 12.60 se

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (VoskAPI:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (VoskAPI:CompileLooped():nnet-compile-looped.cc:345) Spent 0.659921 seconds in looped compilation.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from models/vosk-model-en-us-0.21/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:278) Loading HCLG from models/vosk-model-en-us-0.21/graph/HCLG.fst
LOG (VoskAPI:ReadDataFiles():model.cc:293) Loading words from models/vosk-model-en-

{'once': 'from 3.33 sec to 4.05 sec, confidence is 66.96%'}
{'know': 'from 4.05 sec to 4.74 sec, confidence is 70.88%'}
{'the': 'from 4.83 sec to 5.37 sec, confidence is 100.00%'}
{'family': 'from 5.40 sec to 6.21 sec, confidence is 100.00%'}
{'names': 'from 6.27 sec to 7.20 sec, confidence is 71.36%'}
{'flamingos': 'from 11.85 sec to 13.32 sec, confidence is 50.59%'}
{'are': 'from 13.38 sec to 13.71 sec, confidence is 50.59%'}
{'a': 'from 14.16 sec to 15.03 sec, confidence is 55.97%'}
{'why': 'from 17.04 sec to 17.70 sec, confidence is 74.29%'}
{'to': 'from 20.49 sec to 20.67 sec, confidence is 100.00%'}
{'be': 'from 20.67 sec to 20.94 sec, confidence is 100.00%'}
{'fair': 'from 20.94 sec to 21.60 sec, confidence is 100.00%'}
{'for': 'from 22.41 sec to 22.77 sec, confidence is 100.00%'}
{'car': 'from 22.77 sec to 23.31 sec, confidence is 100.00%'}
{'and': 'from 23.88 sec to 24.14 sec, confidence is 100.00%'}
{'short': 'from 24.15 sec to 24.68 sec, confidence is 44.72%'}
{'the': 'from 

In [211]:
print(audio_word_timestamp.shape)
print(list(audio_word_timestamp['audio_source'].unique()) )
audio_word_timestamp.head()

# save this file into csv 
audio_word_timestamp.to_csv("Targeted_Output/Phase1/0_raw_audio_word_timestamp.csv", index = False)

(1080, 3)
['insects_intro_ant_v', 'birds_intro_tailor_bird_v', 'birds_intro_peacock_v', 'insects_intro_beetle_v', 'birds_intro_flamingo_v']


In [244]:
audio_word_timestamp.head()

,words,timestamps_audio,audio_source
0,oh,"from 14.37 sec to 15.12 sec, confidence is 84.38%",insects_intro_ant_v
1,cool,"from 31.44 sec to 32.25 sec, confidence is 64.88%",insects_intro_ant_v
2,hey,"from 40.50 sec to 41.04 sec, confidence is 100...",insects_intro_ant_v
3,you,"from 58.88 sec to 59.10 sec, confidence is 51.12%",insects_intro_ant_v
4,want,"from 59.10 sec to 59.43 sec, confidence is 51.12%",insects_intro_ant_v


In [ ]:
# backup method to martk words with timestamp
# https://pypi.org/project/SoundFile/
# SoundFile can read and write sound files. File reading/writing is supported through libsndfile, which is a free, 
# cross-platform, open-source (LGPL) library for reading and writing many different sampled sound file formats that runs on many platforms including Windows, OS X, and Unix. 
# It is accessed through CFFI, 
# which is a foreign function interface for Python calling C code. 
# CFFI is supported for CPython 2.6+, 3.x and PyPy 2.0+. SoundFile represents audio data as NumPy arrays.

# import soundfile as sf

# def read_data(annotat, date_format):
#     df = pandas.read_csv(annotat, sep=',')
 
#     # Use proper pandas datatypes
#     df['Time'] = pandas.to_datetime(df['DateTime'], format=date_format)
#     df['Duration'] = pandas.to_timedelta(df['Duration ms'], unit='ms')
#     df['Offset'] = pandas.to_datetime(df['StartOffset ms'], unit='ms')
#     df = df.drop(columns=['DateTime', 'Duration ms', 'StartOffset ms'])
 
#     # Compute start and end time of each segment
#     # audio starts at time of first segment
#     first = df['Time'].iloc[0]
#     df['Start'] = df['Time'] - first
#     df['End'] = df['Start'] + df['Duration']
 
#     return df
 
# def extract_segments(y, sr, segments):
#     # compute segment regions in number of samples
#     starts = numpy.floor(segments.Start.dt.total_seconds() * sr).astype(int)
#     ends = numpy.ceil(segments.End.dt.total_seconds() * sr).astype(int)
 
#     # slice the audio into segments
#     for start, end in zip(starts, ends):
#         audio_seg = y[start:end]
#         print('extracting audio segment:', len(audio_seg), 'samples')
         
# segments = read_data("C:/Users/Mergorine/Audio/annotat.csv", date_format="%m/%d/%y %H:%M:%S:%f")
# print(segments)
 
# y, sr = librosa.load("C:/Users/Mergorine/Audio/audio.wav", sr=16000, duration=2027)
# extract_segments(y, sr, segments)

Convert meta data ppt into text files 

https://stackoverflow.com/questions/13559133/how-to-open-ppt-file-using-python

https://python-pptx.readthedocs.io/en/latest/api/shapes.html#shape-objects-autoshapes



In [220]:
# pip install python-pptx==0.6.21

In [231]:
list_of_ppts

['../../Data/text_univ_chichago/birds_intro_peafowl_t.pptx',
 '../../Data/text_univ_chichago/birds_intro_flamingo_t.pptx',
 '../../Data/text_univ_chichago/insects_intro_ant_t.pptx',
 '../../Data/text_univ_chichago/birds_intro_tailor_bird_t.pptx',
 '../../Data/text_univ_chichago/insects_intro_beetle_t.pptx']

In [229]:
# function to extract all text from the ppt slides
def read_ppt_text(ppt_path):
    from pptx import Presentation
    prs = Presentation(ppt_path)

    # text_runs will be populated with a list of strings,
    # one for each text run in presentation
    text_runs = []

    for slide in prs.slides:
        for shape in slide.shapes:
    #         print(shape)
            if not shape.has_text_frame:
                continue
            for paragraph in shape.text_frame.paragraphs:
                for run in paragraph.runs:
                    text_runs.append(run.text)
    return text_runs

In [232]:

ppts_names = [i.split("/")[-1].split(".")[0] for i in list_of_ppts]
meta_data_lst = [read_ppt_text(ppt_path = i) for i in list_of_ppts]

metadata_dict = dict(zip(ppts_names, meta_data_lst))

In [242]:
raw_metadata = pd.DataFrame(metadata_dict.items(), columns = ['meta_file', "raw_text"])
raw_metadata.head()

,meta_file,raw_text
0,birds_intro_peafowl_t,"[PEAFOWL, Peacock is the National bird of Indi..."
1,birds_intro_flamingo_t,"[Flamingo, Flamingo is about 110 to 150 cm tal..."
2,insects_intro_ant_t,[Ants are one of the most common insects that ...
3,birds_intro_tailor_bird_t,"[Tailorbird, Tailorbird- , The , Tailorbird , ..."
4,insects_intro_beetle_t,[We see many insects around us. Some have wing...


In [243]:
raw_metadata.to_csv("Targeted_Output/Phase1/0_raw_metadata.csv", index = False)